In [32]:
import pandas as pd
import tls_client
from bs4 import BeautifulSoup
from unidecode import unidecode
from betting_functions import get_url_soup
import time

requests = tls_client.Session(
    client_identifier="chrome112",
)


In [33]:
soup = get_url_soup('https://www.pro-football-reference.com/years/2023/games.htm')


In [34]:
script_tag = soup.find('script', type='application/ld+json')
if script_tag:
    json_content = script_tag.string.strip()

In [35]:
import json
data = json.loads(json_content)
df = pd.DataFrame(data)

In [36]:
test_url = df['url'].iloc[0]
test_url = 'https://www.pro-football-reference.com/boxscores/202312030was.htm'

In [37]:
soup = get_url_soup(test_url)

In [ ]:
td_elements = soup.find_all('td', {'data-stat': True})

In [ ]:
tables = soup.find_all('table')

# Iterate through each table and extract data
table_dataframes = []
for index, table in enumerate(tables):
    if index == 2:
        # Convert HTML table to pandas dataframe
        table_df = pd.read_html(str(table))[0]
        table_dataframes.append(table_df)
#table_df.reset_index(inplace=True,drop=True)
table_df.columns = table_df.columns.map(' '.join)
table_df.rename({'Unnamed: 0_level_0 Player':'player',
                 'Unnamed: 1_level_0 Tm':'team' },axis=1,inplace=True)
table_df

,player,team,Passing Cmp,Passing Att,Passing Yds,Passing TD,Passing Int,Passing Sk,Passing Yds.1,Passing Lng,...,Rushing Yds,Rushing TD,Rushing Lng,Receiving Tgt,Receiving Rec,Receiving Yds,Receiving TD,Receiving Lng,Fumbles Fmb,Fumbles FL
0,Tua Tagovailoa,MIA,18,24,280,2,0,0,0,78,...,0,0,0,0,0,0,0,0,0,0
1,Mike White,MIA,1,1,3,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
2,De'Von Achane,MIA,0,0,0,0,0,0,0,0,...,73,2,15,4,3,30,0,21,0,0
3,Raheem Mostert,MIA,0,0,0,0,0,0,0,0,...,43,1,12,1,1,8,0,8,0,0
4,Jeff Wilson,MIA,0,0,0,0,0,0,0,0,...,11,0,6,0,0,0,0,0,0,0
5,Jaylen Waddle,MIA,0,0,0,0,0,0,0,0,...,0,0,0,8,5,52,0,16,0,0
6,Julian Hill,MIA,0,0,0,0,0,0,0,0,...,0,0,0,2,2,23,0,18,0,0
7,River Cracraft,MIA,0,0,0,0,0,0,0,0,...,0,0,0,2,2,16,0,13,0,0
8,Alec Ingold,MIA,0,0,0,0,0,0,0,0,...,0,0,0,1,1,-3,0,-3,0,0
9,Tyreek Hill,MIA,0,0,0,0,0,0,0,0,...,-4,0,6,7,5,157,2,78,0,0


In [ ]:
column_mapping = {
    'Passing Cmp': 'Pass Completions',
    'Passing Att': 'Pass Attempts',
    'Passing Yds': 'Pass Yards',
    'Passing TD': 'Pass TDs',
    'Passing Int': 'Interceptions',
    'Passing Sk': 'Sacks',
    'Passing Yds.1': 'Yards Sacked',
    'Passing Lng': 'Longest Catch',
    'Rushing Att': 'Rush Attempts',
    'Rushing Yds': 'Rush Yards',
    'Rushing TD': 'Rush TDs',
    'Rushing Lng': 'Longest Rush',
    'Receiving Tgt': 'Rec Targets',
    'Receiving Rec': 'Receptions',
    'Receiving Yds': 'Receiving Yards',
    'Receiving TD': 'Rec TDs',
    'Receiving Lng': 'Long Reception',
    'Fumbles Fmb': 'Fumbles',
    'Fumbles FL': 'Fumbles Lost'
}

In [ ]:
table_df.columns = [column_mapping.get(col,col) for col in table_df.columns]

In [ ]:
from game_log import load_current_line
nfl = load_current_line('Lines\pp\pp_2023_12_2.csv','scrape_time')
nfl =pd.read_csv('Lines\pp\pp_2023_12_2.csv',index_col=0)
nfl.loc[nfl.league_name == 'NFL'].stat.unique()

array(['Rec TDs', 'Receptions', 'Receiving Yards', 'Rec Targets', 'Sacks',
       'Tackles+Ast', 'Rush+Rec TDs (Combo)', 'Receiving Yards (Combo)',
       'Pass+Rush+Rec TDs', 'Pass Yards', 'Pass TDs', 'Pass+Rush Yds',
       'Rush Attempts', 'Pass Completions', 'Rush Yards', 'Pass Attempts',
       'Pass Yards (Combo)', 'Rush Yards (Combo)', 'Rush+Rec Yds',
       'FG Made (Combo)', 'FG', 'Kicking Points', 'INT', 'Rush TDs',
       'Punts'], dtype=object)

In [ ]:
from dateutil import parser
date_str = soup.title.text.split(' - ')[-1].split(' | ')[0]
table_df['date'] = parser.parse(date_str)

In [ ]:
teams = table_df['team'].unique()
team_2_opp = {teams[0]:teams[-1],
              teams[-1]:teams[0]}
team_2_opp
table_df['opp'] = table_df['team'].apply(lambda x: team_2_opp.get(x,x))

In [ ]:
nfl.columns

Index(['player', 'team', 'line', 'stat', 'opp', 'league_id', 'league_name',
       'event_time', 'pp_player_id', 'date', 'scrape_time', 'prop_id'],
      dtype='object')

In [ ]:
comb = nfl.merge(table_df,on=['player','team','opp'],how='inner')

In [ ]:
comb['Rec TDs'].unique()

array(['2', '0'], dtype=object)

In [ ]:
df.loc[(df.player=='Tyreek Hill')& (df.stat == 'Receptions')]

,player,team,line,stat,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id
402,Tyreek Hill,MIA,7.0,Receptions,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 21:15:00-06:00,18e2ca0e-5


In [ ]:
def calc_outcome(row):
    outcome = 0
    for part in row['stat'].split('+'):
        try:
            outcome += float(row[part])
        except KeyError:
            return float('nan')
    return outcome

comb['result'] = comb.apply(lambda row: calc_outcome(row),axis=1)
comb['o_u'] = ['under' if x > y else 'over' if x < y else 'push' for x, y in zip(comb['line'], comb['result'])]
comb.drop_duplicates(inplace=True)

In [ ]:
nfl.loc[(nfl.player=='Tyreek Hill')]

,player,team,line,stat,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id
0,Tyreek Hill,MIA,0.5,Rec TDs,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 21:15:00-06:00,8fa6b499-e
1,Tyreek Hill,MIA,7.0,Receptions,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 21:15:00-06:00,18e2ca0e-5
2,Tyreek Hill,MIA,103.5,Receiving Yards,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 21:15:00-06:00,184ad7d5-2
3,Tyreek Hill,MIA,10.5,Rec Targets,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 21:15:00-06:00,a8d4933b-a
0,Tyreek Hill,MIA,0.5,Rec TDs,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 20:45:00-06:00,8fa6b499-e
...,...,...,...,...,...,...,...,...,...,...,...,...
1,Tyreek Hill,MIA,103.5,Receiving Yards,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 10:15:00-06:00,184ad7d5-2
0,Tyreek Hill,MIA,0.5,Rec TDs,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 09:45:00-06:00,8fa6b499-e
1,Tyreek Hill,MIA,103.5,Receiving Yards,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 09:45:00-06:00,184ad7d5-2
0,Tyreek Hill,MIA,0.5,Rec TDs,WAS,9,NFL,2023-12-03 12:00:00-06:00,150711,2023-12-03 12:00:00-06:00,2023-12-02 09:15:00-06:00,8fa6b499-e


In [ ]:
def round_to_bucket(value, bucket_size):
    if bucket_size < 1:
        return value
    else:
        return ((value + bucket_size - 1) // bucket_size) * bucket_size - bucket_size/2
round_to_bucket(5,3)

def graph_history(df,stat,bucket):
    df['stat_hist'] = df['line'].apply(lambda x: round_to_bucket(x,bucket))
    print(stat)
    x = df.loc[df.stat == stat].groupby('stat_hist')['o_u'].value_counts().reset_index().copy(deep=True)
    pivot_df = x.pivot(index='stat_hist', columns='o_u', values='count').fillna(0)
    if 'push' in pivot_df.columns:
        pivot_df['push'].fillna(0,inplace=True)
    else:
        pivot_df['push'] = 0

    if 'over' in pivot_df.columns:
        pivot_df['over'].fillna(0,inplace=True)
    else:
        pivot_df['over'] = 0

    if 'under' in pivot_df.columns:
        pivot_df['under'].fillna(0,inplace=True)
    else:
        pivot_df['under'] = 0
    pivot_df['over_ratio'] =(pivot_df['over']+5)/ (10+pivot_df['under'] + pivot_df['over'])
    pivot_df['count'] = pivot_df['over'] + pivot_df['under'] + pivot_df['push']
    result_df = pivot_df.reset_index()
    over_percent = result_df['over'].sum() / (result_df['under'].sum() + result_df['over'].sum())
    print('overall over %:',over_percent)
    print(f"num of bets: {result_df['count'].sum()}")
    fig, ax1 = plt.subplots()
    ax1.plot(result_df['stat_hist'], result_df['over_ratio'], label='Over Ratio', color='blue')
    ax1.set_xlabel('X-axis Label')
    ax1.set_ylabel('Over Hit Rate', color='blue')

    # Create a second y-axis (right)
    ax2 = ax1.twinx()

    # Plotting the second set of data against the second y-axis (right)
    ax2.bar(result_df['stat_hist'], result_df['count'], label='Count', color='green', alpha=0.5)
    ax2.set_ylabel('Count', color='green')

    # Adding a horizontal line at y = 0.5
    ax1.axhline(y=0.46, color='green', linestyle='-.')
    ax1.axhline(y=0.54, color='green', linestyle='-.')
    ax1.axhline(y=over_percent, color='purple', linestyle='-')
    # Show plot
    plt.show()
    display(pivot_df)
    return

stat = 'ORB'
bucket = 0.5

graph_history(comb,stat,bucket)

ORB
overall over %: nan
num of bets: 0


C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_29576\1508700399.py:30: RuntimeWarning: invalid value encountered in scalar divide
  over_percent = result_df['over'].sum() / (result_df['under'].sum() + result_df['over'].sum())


NameError: name 'plt' is not defined